In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

# Parameters
SPEED_THRESHOLD = 4  # m/s
MIN_STOP_DURATION = 1.0  # seconds
DIST_THRESH = 2.5
EXPORT_INTERVAL = 0.16  # seconds
PED_BOX = (0.3, 0.3)
TUKTUK_BOX = (3.20,1.40)         

# ----------------------------------
# Helpers
# ----------------------------------

def filter_stopped_vehicles(df, speed_col='speed', time_col='TimeStamp'):
    """Filter out stopped vehicles longer than threshold"""
    df = df.copy()
    df['time_diff'] = df.groupby('Track ID')[time_col].diff()
    df['is_stopped'] = df[speed_col] < SPEED_THRESHOLD
    stop_groups = (df['is_stopped'] != df['is_stopped'].shift()).cumsum()
    df['stop_duration'] = df.groupby(['Track ID', stop_groups])['time_diff'].cumsum()
    return df[~((df['is_stopped']) & (df['stop_duration'] >= MIN_STOP_DURATION))].drop(columns=['time_diff', 'is_stopped', 'stop_duration'])

def get_rotated_corners(x, y, heading, length, width):
    """Return coordinates of rotated bounding box corners"""
    half_l, half_w = length / 2, width / 2
    corners = np.array([
        [ half_l,  half_w],
        [ half_l, -half_w],
        [-half_l, -half_w],
        [-half_l,  half_w]
    ])
    rad = np.radians(heading)
    rot = np.array([
        [np.cos(rad), -np.sin(rad)],
        [np.sin(rad),  np.cos(rad)]
    ])
    return (corners @ rot.T) + np.array([x, y])

def get_closest_corners(row):
    """Get direction vector and min distance between closest corners"""
    ped_corners = get_rotated_corners(row['x_smooth_ped'], row['y_smooth_ped'], row['HA_ped'], *PED_BOX)
    tuktuk_corners = get_rotated_corners(row['x_smooth_tuktuk'], row['y_smooth_tuktuk'], row['HA_tuktuk'], *TUKTUK_BOX)
    dists = cdist(ped_corners, tuktuk_corners)
    idx = np.unravel_index(np.argmin(dists), dists.shape)
    min_dist = dists[idx]
    direction_vec = tuktuk_corners[idx[1]] - ped_corners[idx[0]]
    return min_dist, direction_vec

def calculate_attc(row):
    """Compute ATTC using projection of relative motion on direction vector"""
    min_dist, direction_vec = get_closest_corners(row)
    norm = np.linalg.norm(direction_vec)
    if norm == 0:
        return np.inf
    unit_vec = direction_vec / norm

    rel_v = np.array([
        row['vx_smooth_ped'] - row['vx_smooth_tuktuk'],
        row['vy_smooth_ped'] - row['vy_smooth_tuktuk']
    ])
    rel_a = 0.5 * np.array([
        row['ax_ped'] - row['ax_tuktuk'],
        row['ay_ped'] - row['ay_tuktuk']
    ])

    closing_rate = -np.dot(rel_v, unit_vec) + np.dot(rel_a, unit_vec)
    return min_dist / closing_rate if closing_rate > 0 else np.inf

# ----------------------------------
# Load Data
# ----------------------------------

df_ped = pd.read_csv(r"D:\T\test_codeEVT\nd\ped_smooth.csv")
df_tuktuk = pd.read_csv(r"D:\T\test_codeEVT\nd\tuktuk_smooth.csv")
df_tuktuk['speed'] = np.sqrt(df_tuktuk['vx_smooth']**2 + df_tuktuk['vy_smooth']**2)

# Filter stopped vehicles
print(f"Original tuktukrcycle count: {len(df_tuktuk)}")
df_tuktuk = filter_stopped_vehicles(df_tuktuk, speed_col='speed')
print(f"Filtered tuktukrcycle count: {len(df_tuktuk)}")
print(df_tuktuk['speed'].mean())
# Calculate yaw rates
for df in [df_ped, df_tuktuk]:
    df['yaw_rate'] = df.groupby('Track ID').apply(
        lambda x: x['HA'].diff() / x['TimeStamp'].diff()
    ).reset_index(level=0, drop=True)

# Round timestamps
df_ped['Time_rounded'] = (df_ped['TimeStamp'] / EXPORT_INTERVAL).round() * EXPORT_INTERVAL
df_tuktuk['Time_rounded'] = (df_tuktuk['TimeStamp'] / EXPORT_INTERVAL).round() * EXPORT_INTERVAL


Original tuktukrcycle count: 379048
Filtered tuktukrcycle count: 71582
5.983126558990913


C:\Users\IST LAB 1\AppData\Local\Temp\ipykernel_8560\3620123155.py:87: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['yaw_rate'] = df.groupby('Track ID').apply(
C:\Users\IST LAB 1\AppData\Local\Temp\ipykernel_8560\3620123155.py:87: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['yaw_rate'] = df.groupby('Track ID').apply(


In [3]:

# ----------------------------------
# Merge and Process
# ----------------------------------

merged = pd.merge(
    df_ped, 
    df_tuktuk, 
    on='Time_rounded', 
    suffixes=('_ped', '_tuktuk')
)

merged['Center_dist'] = np.hypot(
    merged['x_smooth_ped'] - merged['x_smooth_tuktuk'],
    merged['y_smooth_ped'] - merged['y_smooth_tuktuk']
)

# Filter by center threshold
results = merged[merged['Center_dist'] <= DIST_THRESH].copy()

# Calculate ATTC using corrected method
results['ATTC'] = results.apply(calculate_attc, axis=1)
results[['mindis', 'direction_vec']] = results.apply(
    lambda row: pd.Series(get_closest_corners(row)),
    axis=1
)
# Output
output = results[['Track ID_ped', 'Track ID_tuktuk', 'TimeStamp_ped', 'ATTC']].rename(columns={
    'Track ID_ped': 'Ped_ID',
    'Track ID_tuktuk': 'tuktuk_ID',
    'TimeStamp_ped': 'TimeStamp'
})
print(output)

len(results[(results['ATTC']<1) & (results['mindis']<1)][['Track ID_ped', 'Track ID_tuktuk']].drop_duplicates())

        Ped_ID  tuktuk_ID  TimeStamp      ATTC
4337       382        389     122.84       inf
4342       382        389     123.00       inf
4348       382        389     123.16       inf
4352       382        389     123.32  2.584227
4601       391        389     122.96       inf
...        ...        ...        ...       ...
338114   15742      15744    5403.84  0.324089
342590   15849      15809    5428.60  0.209432
342593   15849      15809    5428.76       inf
348598   16063      16040    5486.44  0.189668
348602   16063      16040    5486.60       inf

[1437 rows x 4 columns]


115

In [4]:
results[results['ATTC']<1][['Track ID_ped', 'Track ID_tuktuk']].drop_duplicates()

,Track ID_ped,Track ID_tuktuk
4607,391,389
9636,1067,1053
11212,1168,1207
11256,1168,1212
16041,1311,1080
...,...,...
325485,15497,15424
335632,15693,15753
338114,15742,15744
342590,15849,15809


In [9]:
# Create new DataFrame with selected and renamed columns
conflict_summary = results[[
    'Track ID_ped',        # Pedestrian ID
    'Track ID_tuktuk',       # Vehicle ID
    'Type_tuktuk',           # Vehicle type
    'TimeStamp_ped',       # Timestamp
    'mindis',              # Minimum corner distance
    'ATTC'                 # ATTC value
]].copy()

# Rename columns
conflict_summary.rename(columns={
    'Track ID_ped': 'PED_ID',
    'Track ID_tuktuk': 'VEH_ID',
    'Type_tuktuk': 'TYPE',
    'TimeStamp_ped': 'TIMESTAMP',
    'mindis': 'MIN_COR_DIS',
    'ATTC': 'ATTC'
}, inplace=True)

# Preview
conflict_summary['TYPE'] = conflict_summary['TYPE'].replace(' Tuk-Tuk', 'Auto-Rikshaw')
print(conflict_summary.head())


      PED_ID  VEH_ID          TYPE  TIMESTAMP  MIN_COR_DIS      ATTC
4337     382     389  Auto-Rikshaw     122.84     0.693502       inf
4342     382     389  Auto-Rikshaw     123.00     0.724449       inf
4348     382     389  Auto-Rikshaw     123.16     0.740446       inf
4352     382     389  Auto-Rikshaw     123.32     0.447033  2.584227
4601     391     389  Auto-Rikshaw     122.96          NaN       inf


In [10]:
conflict_summary.to_csv(r"D:\T\test_codeEVT\ATTC_Data/tuktuk_Ped.csv", index=False)